## Ironhack Data Analytics Bootcamp

*Nayely Saldivar*

#### Proyectos: 
* Web Scrapping y Data Cleaning


In [1]:
# Importamos las librerías necesarias
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from bs4 import BeautifulSoup
import time

### Definimos las funciones que usaremos para obtener información del Portal de Fraudes

In [2]:
def FirstStep():
    
    # Guardamos nuestro sitio en la variable url
    url = 'https://phpapps.condusef.gob.mx/fraudes_financieros/monitor.php'
    
    return url

In [3]:
def Collect_Path(browser,keys):
    # Buscamos por nuestra llave
    buscador = browser.find_element_by_id('searchField')
    buscador.send_keys(keys)
    boton = browser.find_element_by_name('busca').click()
    time.sleep(5)

    #Recolectamos todos los registros por el path de las publicaciones
    registros=browser.find_elements_by_xpath('//*[@id="tab_for"]')
    return registros

In [4]:
def Collect_Data(browser,registros):
    
    # Definimos las estructuras
    Fecha_Reporte=[]
    Dato_f=[]
    Institucion_Afec=[]
    Tipo_f=[]
    Fuente=[]  
    
    # Recolectamos los datos con la estructura que encontramos
    # ---Fechas
    Rec_Fechas=registros[0].find_elements_by_tag_name('strong')[1:]
    Fecha_Reporte=[Rec_Fechas[i].text for i in range(len(Rec_Fechas))]

    # --- Dato fraudulento
    Rec_Dato=registros[0].find_elements_by_css_selector('h3:nth-child(2)')
    Dato_f=[Rec_Dato[i].text for i in range(len(Rec_Dato))]

    # ---Instituciones Financieras
    Rec_Ins=registros[0].find_elements_by_css_selector('h3:nth-child(5)')
    Institucion_Afec=[Rec_Ins[i].text for i in range(len(Rec_Ins))]

    # --- El tipo de fraude
    Rec_Tipo=registros[0].find_elements_by_css_selector('h4:nth-child(8)')
    Tipo_f=[Rec_Tipo[i].text for i in range(len(Rec_Tipo))]

    # --- La institución que valida el fraude
    Rec_fuente=registros[0].find_elements_by_css_selector('h4:nth-child(11)')
    Fuente=[Rec_fuente[i].text for i in range(len(Rec_fuente))]
    
    # Creamos el dataframe con la información recolectada
    data=[Fecha_Reporte,Dato_f,Institucion_Afec,Tipo_f,Fuente]

    return data

In [5]:
def Create_DF(data):
    
#   Creamos el dataframe con la información recolectada
    Base=pd.DataFrame(data).transpose()
    Base.columns=['Fecha','Dato Fraudulento','Institucion Afectada','Tipo de Fraude','Validacion']

    return Base

Se define la función principal del scrapper : **Portal_Fraude_Scrapper**

In [6]:
def Portal_Fraude_Scrapper(key):
    
    url=FirstStep()
    
    # Iniciamos el driver
    browser = webdriver.Chrome()
    browser.get(url)
    
    time.sleep(5)
    
    # Iniciamos las funciones
    reports=Collect_Path(browser,key)
    print(f'Conexión completa, buscando:{key}')
    time.sleep(1)
    
    data=Collect_Data(browser,reports)
    print('Recolección de datos completa')
    time.sleep(1)
    
    df_base=Create_DF(data)
    print(f'Se completó el proceso, key:{key}')
    
    # Cerramos el driver    
    browser.quit()
    
    return df_base

### Recolectamos información por las llaves que nos interesan

In [7]:
# Con base a mi proyecto final, recolectaré la información que arroje buscar 'face' y 'redes' en el sitio
keys=['face','redes']

In [8]:
df_face=Portal_Fraude_Scrapper(keys[0])
df_face.head()

Conexión completa, buscando:face
Recolección de datos completa
Se completó el proceso, key:face


,Fecha,Dato Fraudulento,Institucion Afectada,Tipo de Fraude,Validacion
0,2020-11-09,Facebook: Impulso Economico,"Impulso Economico Activo, S.A. de C.V., SOFOM,...",Suplantación de Institución Financiera,INSTITUCIONES FINANCIERAS
1,2020-11-05,Facebook: Creavanza,"Grupo Asesor Creavanza, S.A.P.I. de C.V., SOFO...",Suplantación de Institución Financiera,INSTITUCIONES FINANCIERAS
2,2020-11-05,Facebook: Cre-a-vanza,"Grupo Asesor Creavanza, S.A.P.I. de C.V., SOFO...",Suplantación de Institución Financiera,INSTITUCIONES FINANCIERAS
3,2020-10-29,Facebook: CrediLeal,"Emprende Capital, S.A.P.I. de C.V., SOFOM, E.N.R.",Suplantación de Institución Financiera,INSTITUCIONES FINANCIERAS
4,2020-10-28,"Facebook: Patrimoniales, S.A.P.I. de CV SOFOM ENR","Soluciones Patrimoniales Cufrisa, S.A. de C.V....",Suplantación de Institución Financiera,INSTITUCIONES FINANCIERAS


In [10]:
df_redes=Portal_Fraude_Scrapper(keys[1])
df_redes.head()

Conexión completa, buscando:redes
Recolección de datos completa
Se completó el proceso, key:redes


,Fecha,Dato Fraudulento,Institucion Afectada,Tipo de Fraude,Validacion
0,2020-02-28,@BBVARe_bmx,"BBVA Bancomer, S.A., Institución de Banca Múlt...",Redes sociales,Portal de Fraudes Financieros
1,2020-02-27,https://www.facebook.com/103074507769643/posts...,"Caja Solidaria San Sebastián del Oeste, S.C. d...",Redes Sociales,INSTITUCIONES FINANCIERAS
2,2020-02-27,https://www.facebook.com/naomi.altunar.7?fref=...,"Crediclub, S.A. de C.V., S.F.P.",Redes sociales,Portal de Fraudes Financieros
3,2020-02-24,https://www.facebook.com/colectivofinancierona...,Sin dato preciso,Redes sociales,Portal de Fraudes Financieros
4,2020-02-20,https://www.auditoriacondusef.com,CONDUSEF,Página de internet (Pharming),Redes Sociales Twitter CONDUSEF


### Definimos las funciones que usaremos para limpiar la información obtenida

In [11]:
def ConcatDF(df,df1):
    df_complete=pd.concat([df,df1],axis=0)
    print('Elementos en df_1:',len(df))
    print('Elementos en df_2:',len(df1))
    print('Elementos totales en el nuevo df:',len(df_complete))
    return df_complete

In [12]:
def limpiar_col(x):
    reemplazar = [('-', ''), ('á', 'a'), ('é', 'e'),
                  ('ó', 'o'), ('ú','u'), ('í','i')]
    for tupla in reemplazar:
        x = x.lower().replace(tupla[0], tupla[1])
    return x

In [14]:
def CleaningDF(df):
    
    if df.isnull().sum().sum() !=0:
        df.fillna('sin registro')
    
    # Transformamos a minúsculas y eliminamos acentos    
    for col in df.columns[1:]:
        df[col] = df[col].apply(limpiar_col)
    
    # Eliminamos los registros duplicados
    antes=len(df)
    df_complete = df.drop_duplicates()
    despues=len(df_complete)
    print('Elementos duplicados eliminados:',(antes-despues))
    
    # Creamos una columna de trimestres  
    df_complete['Fecha']=pd.to_datetime(df_complete['Fecha'])
    df_complete['Q']=df_complete['Fecha'].dt.to_period('Q')
    return df_complete

In [15]:
def FilteringDF(df):
    
    df_filtered=df[(df['Tipo de Fraude']==('redes sociales')) | 
                   (df['Tipo de Fraude']=='suplantacion de institucion financiera')]
   
    print('Registros por Fraudes en Redes Sociales:',len(df_filtered))
    
    return df_filtered

Se define la función principal del scrapper : **Portal_Fraude_CleaningPipe**

In [16]:
def Portal_Fraude_CleaningPipe(df,df1):
    
    df_complete=ConcatDF(df,df1)
    time.sleep(1)
    
    df_clean=CleaningDF(df_complete)
    time.sleep(1)
    
    df_filtered=FilteringDF(df_clean)
    time.sleep(1)
    
    # Exportamos el reporte en csv    
    df_filtered.to_csv(f'MonitoreoCONDUSEF_Keys({keys[0].capitalize()}-{keys[1].capitalize()}).csv',index=False)
    time.sleep(1)
    
    print('Se exportó el reporte en CSV :)')

    return df_filtered

### Unimos las bases, limpiamos los datos y unificamos la información

In [17]:
df=Portal_Fraude_CleaningPipe(df_face,df_redes)

Elementos en df_1: 97
Elementos en df_2: 198
Elementos totales en el nuevo df: 295
Elementos duplicados eliminados: 70


<ipython-input-14-24e4d99c9b1e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complete['Fecha']=pd.to_datetime(df_complete['Fecha'])
<ipython-input-14-24e4d99c9b1e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complete['Q']=df_complete['Fecha'].dt.to_period('Q')


Registros por Fraudes en Redes Sociales: 202
Se exportó el reporte en CSV :)


#### Realizamos algunas agregaciones y tablas

In [18]:
df.head()

,Fecha,Dato Fraudulento,Institucion Afectada,Tipo de Fraude,Validacion,Q
0,2020-11-09,facebook: impulso economico,"impulso economico activo, s.a. de c.v., sofom,...",suplantacion de institucion financiera,instituciones financieras,2020Q4
1,2020-11-05,facebook: creavanza,"grupo asesor creavanza, s.a.p.i. de c.v., sofo...",suplantacion de institucion financiera,instituciones financieras,2020Q4
3,2020-10-29,facebook: credileal,"emprende capital, s.a.p.i. de c.v., sofom, e.n.r.",suplantacion de institucion financiera,instituciones financieras,2020Q4
4,2020-10-28,"facebook: patrimoniales, s.a.p.i. de cv sofom enr","soluciones patrimoniales cufrisa, s.a. de c.v....",suplantacion de institucion financiera,instituciones financieras,2020Q4
5,2020-10-16,facebook: financiera logros economicos s.a de c.v,"grt logros, s.a. de c.v., sofom, e.n.r.",suplantacion de institucion financiera,instituciones financieras,2020Q4


In [19]:
categorical_columns=[c for c in df.columns if df[c].dtype=="object"]
[f"Valores unicos en {col}: {len(df[col].unique())}" for col in categorical_columns ]

['Valores unicos en Dato Fraudulento: 178',
 'Valores unicos en Institucion Afectada: 100',
 'Valores unicos en Tipo de Fraude: 2',
 'Valores unicos en Validacion: 3']

In [20]:
Fuentes_Validacion = (
    df
      .groupby([ "Validacion"])
      [["Dato Fraudulento"]]
      .count()
)
Fuentes_Validacion

,Dato Fraudulento
Validacion,
condusef,20
instituciones financieras,53
portal de fraudes financieros,129


In [21]:
Tipo_Fraude = (
    df
      .groupby([ "Tipo de Fraude"])
      [["Dato Fraudulento"]]
      .count()
)
Tipo_Fraude

,Dato Fraudulento
Tipo de Fraude,
redes sociales,186
suplantacion de institucion financiera,16


In [22]:
Repor_Trimestral = (
    df
      .groupby([ "Q"])
      [["Dato Fraudulento"]]
      .count()
)
Repor_Trimestral

,Dato Fraudulento
Q,
2018Q1,10
2018Q2,4
2018Q3,3
2018Q4,74
2019Q1,41
2019Q2,14
2019Q3,4
2019Q4,23
2020Q1,13


In [24]:
Fraude_e_Institucion = (
    df
      .groupby(["Tipo de Fraude","Validacion"])
      .agg(
          reportes = ("Dato Fraudulento", "count"))
      )
Fraude_e_Institucion

reportes
Tipo de Fraude                         Validacion                             
redes sociales                         condusef                             20
                                       instituciones financieras            37
                                       portal de fraudes financieros       129
suplantacion de institucion financiera instituciones financieras            16